# Feagin's Order 10, 12, and 14 Methods

DifferentialEquations.jl includes Feagin's explicit Runge-Kutta methods of orders 10/8, 12/10, and 14/12. These methods have such high order that it's pretty much required that one uses numbers with more precision than Float64. As a prerequisite reference on how to use arbitrary number systems (including higher precision) in the numerical solvers, please see the Solving Equations in With Chosen Number Types notebook.

## Investigation of the Method's Error

We can use Feagin's order 16 method as follows. Let's use the twoDimlinearODEExample from previous examples. Like in the Solving Equations in With Chosen Number Types notebook, we change the initial condition to BigFloats to tell the solver to use BigFloat types.

In [23]:
using DifferentialEquations
prob = twoDimlinearODEExample(α=ones(BigFloat,4,2),u₀=map(BigFloat,rand(4,2)).*ones(4,2)/2)
sol =solve(prob::ODEProblem,Δt=1//16,alg=:Feagin14,adaptive=false);

Let's print the errors and find out how accurate it is:

In [24]:
print(sol.errors)

Dict(:l∞=>1.068447416044847995153951815667497622158599400262425431935379392038731834474638e-23,:final=>5.864524355090247017965494838083295718647660536333265288958331216674935185099171e-24,:l2=>3.284324318110724241546644303457557201807515293900243891513074242871624143202249e-24)

Compare that to machine $\epsilon$ for Float64:

In [8]:
eps(Float64)

2.220446049250313e-16

The error for Feagin's method when the stepsize is 1/16 is 8 orders of magnitude below machine $\epsilon$! However, that is dependent on the stepsize. If we instead use adaptive timestepping with the default tolerances [and set the internalnorm=1 to avoid [this issue](https://github.com/JuliaLang/julia/issues/17728)], we get

In [22]:
sol =solve(prob::ODEProblem,Δt=Δts[1],alg=:Feagin14,adaptive=true,internalnorm=1); 
println(sol.errors); print("The length was $(length(sol))")

Dict(:l∞=>5.801079887044034925501010902839807528274531536176388596639490179927086540236457e-01,:final=>3.712067380360646564271528475755090491548152659265300874046301204290046476491424e-01,:l2=>1.923369284386002215322244764769691922687112963476072043780302286755884157640694e-01)
The length was 7

Notice that when the stepsize is much higher, the error goes up quickly as well. These super high order methods are best when used to gain really accurate approximations (using still modest timesteps). Some examples of where such precision is necessary is astrodynamics where the many-body problem is highly chaotic and thus sensitive to small errors.

## Convergence Test

The Order 14 method is awesome, but we need to make sure it's really that awesome. The following convergence test is used in the package tests in order to make sure the implementation is correct. Note that all methods have such tests in place.

In [29]:
Δts = 1.//2.^(10:-1:4)
sim = test_convergence(Δts,prob,alg=:Feagin14)

DifferentialEquations.ConvergenceSimulation of length 7.
Convergence Estimates: (l∞,14.290261124780194) (final,14.290261124780194) (l2,14.29968233607698)

For a better view of what's going on, let's plot the simulation results.

In [31]:
plot(sim)

This is a clear trend indicating that the convergence is truely Order 14. 